In [1]:
import pandas as pd
import numpy as np
import pathlib
titles = pd.read_csv(
        pathlib.Path('./data/title.basics.tsv').resolve(),
        delimiter='\t',
        index_col=['tconst'],
        na_values=['\\N'],
        low_memory=False
)
ratings = pd.read_csv(
        pathlib.Path('./data/title.ratings.tsv').resolve(),
        delimiter='\t',
        index_col=['tconst'],
        na_values=['\\N'],
        low_memory=False
)

# Combine data-sources on 'tconst' (works like left inner join)
df = pd.merge(titles, ratings, on="tconst")

# Clean up runtimeMinutes data
df['runtimeMinutes'] = df['runtimeMinutes'].fillna(0)
df = df[df['runtimeMinutes'].str.isnumeric().astype('bool')].copy()
df['runtimeMinutes'] = pd.to_numeric(df['runtimeMinutes'])

# StartYear should be int, not float

# genres are comma sep list, turned into a list
df['genre_list'] = df['genres'].str.split(',')

# explode out based on genre (to new dataframe) for easier genre lookup
df_by_genre = df.explode(['genre_list']).copy()

df.head(5)
df_by_genre.head(5)

,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes,genre_list
tconst,,,,,,,,,,,
tt0000001,short,Carmencita,Carmencita,0.0,1894.0,NaN,1,"Documentary,Short",5.7,2131,Documentary
tt0000001,short,Carmencita,Carmencita,0.0,1894.0,NaN,1,"Documentary,Short",5.7,2131,Short
tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0.0,1892.0,NaN,5,"Animation,Short",5.6,289,Animation
tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0.0,1892.0,NaN,5,"Animation,Short",5.6,289,Short
tt0000003,short,Poor Pierrot,Pauvre Pierrot,0.0,1892.0,NaN,5,"Animation,Comedy,Romance",6.4,2167,Animation


In [40]:
del df
del df_by_genre
del ratings
del titles


In [83]:
# %timeit df_by_genre.iloc[np.where(df_by_genre['genre_list'] == "Horror")] - slightly faster using numpy
# %timeit df_by_genre[(df_by_genre.genre_list == 'Horror')]

# using np.where for bool ops is faster
filtered_data = df_by_genre.iloc[
    np.where(
        ( df_by_genre.titleType == 'movie') & 
        ( df_by_genre.genre_list == 'Horror') & 
        ( df_by_genre.numVotes > 15000 ))
    ].copy()

filtered_data = filtered_data.sort_values(by='averageRating', ascending=False)
filtered_data[:100]


,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes,genre_list
tconst,,,,,,,,,,,
tt0102926,movie,The Silence of the Lambs,The Silence of the Lambs,0.0,1991.0,NaN,118,"Crime,Drama,Horror",8.6,1614239,Horror
tt0078748,movie,Alien,Alien,0.0,1979.0,NaN,117,"Horror,Sci-Fi",8.5,1004795,Horror
tt0054215,movie,Psycho,Psycho,0.0,1960.0,NaN,109,"Horror,Mystery,Thriller",8.5,745372,Horror
tt0090605,movie,Aliens,Aliens,0.0,1986.0,NaN,137,"Action,Adventure,Horror",8.4,801900,Horror
tt0081505,movie,The Shining,The Shining,0.0,1980.0,NaN,146,"Drama,Horror",8.4,1154392,Horror
...,...,...,...,...,...,...,...,...,...,...,...
tt0408236,movie,Sweeney Todd: The Demon Barber of Fleet Street,Sweeney Todd: The Demon Barber of Fleet Street,0.0,2007.0,NaN,116,"Drama,Horror,Musical",7.3,395643,Horror
tt7784604,movie,Hereditary,Hereditary,0.0,2018.0,NaN,127,"Drama,Horror,Mystery",7.3,411289,Horror
tt0076162,movie,House,Hausu,0.0,1977.0,NaN,88,"Comedy,Horror",7.3,36195,Horror


In [3]:
df_by_genre['startYear'].value_counts()

startYear
2019.0    129105
2022.0    129080
2018.0    126294
2017.0    125637
2021.0    124924
           ...  
1882.0         4
1889.0         3
1874.0         2
1883.0         2
1885.0         2
Name: count, Length: 145, dtype: int64

In [6]:
df.isnull().sum()

titleType               0
primaryTitle            1
originalTitle           1
isAdult                 0
startYear             248
endYear           1474456
runtimeMinutes          0
genres              21731
averageRating           0
numVotes                0
genre_list          21731
dtype: int64

In [18]:
# data_indexed.pivot_table('Revenue (Millions)', index='Director', aggfunc='sum', columns='Year').head(10)

df_by_genre.pivot_table('originalTitle', index='genre_list', aggfunc='count', columns='startYear').dropna(axis=1, thresh=20)


startYear,1905.0,1906.0,1908.0,1909.0,1910.0,1911.0,1912.0,1913.0,1914.0,1915.0,...,2016.0,2017.0,2018.0,2019.0,2020.0,2021.0,2022.0,2023.0,2024.0,2025.0
genre_list,,,,,,,,,,,,,,,,,,,,,
Action,3.0,2.0,21.0,1.0,2.0,4.0,6.0,8.0,13.0,19.0,...,6917.0,7451.0,7741.0,7881.0,6665.0,7877.0,7933.0,7587.0,6296.0,635.0
Adult,NaN,NaN,1.0,1.0,1.0,NaN,NaN,NaN,1.0,1.0,...,504.0,445.0,378.0,337.0,241.0,178.0,126.0,69.0,42.0,NaN
Adventure,3.0,1.0,4.0,4.0,8.0,13.0,15.0,18.0,41.0,62.0,...,5834.0,6646.0,6719.0,6613.0,5800.0,6340.0,6358.0,5749.0,5550.0,506.0
Animation,1.0,2.0,16.0,17.0,18.0,6.0,11.0,9.0,6.0,21.0,...,6965.0,7445.0,7762.0,7679.0,6764.0,7243.0,6582.0,6794.0,5713.0,484.0
Biography,1.0,2.0,2.0,3.0,5.0,4.0,5.0,4.0,4.0,4.0,...,1548.0,1580.0,1566.0,1603.0,1307.0,1311.0,1945.0,1505.0,1068.0,61.0
Comedy,57.0,76.0,87.0,100.0,106.0,149.0,252.0,200.0,183.0,242.0,...,18904.0,19373.0,18955.0,19152.0,17267.0,17745.0,17809.0,15619.0,12911.0,1271.0
Crime,14.0,10.0,11.0,11.0,5.0,9.0,21.0,15.0,22.0,23.0,...,6133.0,6509.0,6741.0,7236.0,6404.0,7452.0,7763.0,7070.0,5513.0,576.0
Documentary,69.0,96.0,31.0,28.0,41.0,37.0,43.0,26.0,11.0,21.0,...,8870.0,9277.0,9828.0,9826.0,9660.0,9128.0,9273.0,7957.0,6404.0,417.0
Drama,31.0,45.0,101.0,165.0,170.0,221.0,258.0,213.0,207.0,210.0,...,18446.0,19780.0,20397.0,21130.0,18964.0,21217.0,23157.0,21878.0,17987.0,1736.0
